<a href="https://colab.research.google.com/github/strongeryoung/class_SmartDataAnalysis/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Stock_Investment_Strategy/Function')

FileNotFoundError: ignored

In [ ]:
pip install xmltodict

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
from KEYS import *
import pickle


# coding: utf-8
import requests 	  # API 요청할 때 쓰는 라이브러리
import xmltodict 	  # xml형태를 dictionary {'a':1,'b':23}
import json  # json을 쓰기위함
import os
import bs4

from datetime import *
from urllib.parse import urlencode, quote_plus, unquote

import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from scipy import stats

from statsmodels.tsa.stattools import adfuller

ModuleNotFoundError: ignored

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm


%matplotlib inline
from matplotlib import rcParams
from matplotlib import font_manager, rc
import matplotlib.dates as mdates
import seaborn as sns

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# [출처] [파이썬] colab에서 plt 한글깨짐 해결|작성자 서울시립대 통계학과

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔글꼴 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_prop = fm.FontProperties(fname=font_path)

plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['font.size'] = 12
# [출처] [파이썬] colab에서 plt 한글깨짐 해결|작성자 서울시립대 통계학과

In [ ]:
## 정상화 테스트

def stationarity_adf_test(Y_Data, col_name):
    if len(col_name) == 0:
        Stationarity_adf = pd.Series(sm.tsa.stattools.adfuller(Y_Data)[0:2],
                                     index=['Statistics', 'p_value'])
        for key, value in sm.tsa.stattools.adfuller(Y_Data)[4].items():
            Stationarity_adf['Critical Value(%s)'%key] = value
            Stationarity_adf['Maximum Information Criteria'] = sm.tsa.stattools.adfuller(Y_Data)[5]
            Stationarity_adf = pd.DataFrame(Stationarity_adf, columns=['Stationarity_adf'])
    else:
        Stationarity_adf = pd.Series(sm.tsa.stattools.adfuller(Y_Data[col_name])[0:2],
                                     index=['Statistics', 'p_value'])
        for key, value in sm.tsa.stattools.adfuller(Y_Data[col_name])[4].items():
            Stationarity_adf['Critical Value(%s)'%key] = value
            Stationarity_adf['Maximum Information Criteria'] = sm.tsa.stattools.adfuller(Y_Data[col_name])[5]
            Stationarity_adf = pd.DataFrame(Stationarity_adf, columns=['Stationarity_adf'])

    return Stationarity_adf


def get_time_series_for_bok(y, m, d, adj_days):

    today = date.today()

    # 경제 데이터 발표 주기를 고려한 날짜 조정 : 1개월 빼기
    adj_dates = adj_days

    time_adj = today - timedelta(days=adj_dates)
    s_date = date(y, m, d)

    # 텍스트화된 날자 데이터 만들기
    date1 = s_date.strftime("%Y-%m-%d").replace('-', '')
    date2 = time_adj.strftime("%Y-%m-%d").replace('-', '')

    #date2 = str(date2)

    # 시작 날짜와 끝 날짜를 datetime 형태로 변환
    start_date = datetime.strptime(date1, "%Y%m%d")
    end_date = datetime.strptime(date2, "%Y%m%d")

    # date_range를 사용하여 time series 인덱스 생성
    time_index = pd.date_range(start=start_date, end=end_date, freq='M')

    n = len(time_index)

    start_dt = date1[:-2]
    end_dt = date2[:-2]

    return start_dt, end_dt, time_index, n


## 월간 경제 데이터 수신을 위한 API 함수 만들기

def get_m_econ_data(n, item_code1, item_code2, item_code3, date1, date2, inter):

    df_list = []

    print(item_code1, item_code2)

    key = KEYS['BOK']
    url =  f"http://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/{n}/{item_code1}/{inter}/{date1}/{date2}/{item_code2}/{item_code3}"
    response = requests.get(url)
    #print(response.status_code)
    data = response.text
    data = json.loads(data)
    data_df = pd.DataFrame(data['StatisticSearch']['row'])
    #print(data_df)
    df_list.append(data_df)

    return df_list


def get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, s_date, e_date, inter):

    df_list = []

    date1 = s_date[:-2]
    date2 = e_date[:-2]

    for item in item_code2:

        temp_df = get_m_econ_data(n, item_code1, item, item_code3, s_date, e_date, inter)
        temp_df = temp_df[0]

        temp_df['DATA_VALUE'] = temp_df['DATA_VALUE'].astype('float64')
        temp_df['interval'] = 'M'
        temp_df = temp_df[['TIME', 'ITEM_NAME1', 'DATA_VALUE', 'interval']].copy()

        # 'TIME' 열을 날짜 형식으로 변환
        temp_df['TIME'] = pd.to_datetime(temp_df['TIME'], format='%Y%m')
        # 월말 기준으로 'TIME' 열을 변환
        temp_df['TIME'] = temp_df['TIME'] + pd.offsets.MonthEnd(0)
        temp_df = temp_df.set_index('TIME')
        temp_df.index.name = 'Date'
        df_list.append(temp_df)

    df = pd.concat(df_list)

    return df


def get_bok_daily_data(n, item_code1, item_code2, s_date, e_date):

    df_list = []

    for item in item_code2:

        temp_df = get_d_econ_data (n, item_code1, item, date1=s_date, date2=e_date)
        temp_df = temp_df[0]
        temp_df['DATA_VALUE'] = temp_df['DATA_VALUE'].astype('float64')
        temp_df = temp_df[['TIME', 'ITEM_NAME1', 'DATA_VALUE']]
        temp_df['interval'] = 'D'
        stock_df = temp_df [['TIME', 'ITEM_NAME1', 'DATA_VALUE', 'interval']]
        df_list.append(stock_df)

    df = pd.concat(df_list)

    return df

def get_d_econ_data (n, item_code1, item_code2, date1, date2):

    df_list = []

    # print(item_code1, item_code2)

    key =  KEYS['BOK']
    url = f"http://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/{n}/{item_code1}/D/{date1}/{date2}/{item_code2}/"

    response = requests.get(url)
    #print(response.status_code)
    data = response.text
    data = json.loads(data)
    #print(data)
    data_df = pd.DataFrame(data['StatisticSearch']['row'])

    df_list.append(data_df)
    #df = pd.concat(df_list)
    return df_list

# Principal Component Analysis (PCA)란?

주성분 분석(Principal Component Analysis, PCA)은 다차원 데이터의 차원을 줄이고 데이터의 주요한 구조를 보존하는데 사용되는 통계적인 방법입니다. 주로 데이터의 차원을 감소시키면서 정보 손실을 최소화하거나 데이터의 노이즈를 제거하기 위해 적용됩니다. PCA는 데이터를 새로운 좌표계로 변환하여 기존 데이터의 특성을 최대한 유지하면서 차원을 축소하는 방법으로 알려져 있습니다.

PCA의 주요 단계는 다음과 같습니다:

1. **데이터 중심화 (Centering):** 각 특성에서 평균을 빼서 데이터를 중심화합니다. 이렇게 하면 데이터의 중심이 원점에 위치하게 됩니다.

2. **공분산 행렬 계산:** 중심화된 데이터의 공분산 행렬을 계산합니다. 공분산은 두 변수 간의 상관 관계를 나타내며 PCA는 데이터의 상관 관계를 이용하여 주성분을 찾습니다.

3. **고유값과 고유벡터 계산:** 공분산 행렬의 고유값(eigenvalue)과 고유벡터(eigenvector)를 계산합니다. 고유값은 각 고유벡터에 대응되는 스칼라 값으로, 주성분의 중요도를 나타냅니다.

4. **고유값 정렬:** 고유값을 크기순으로 정렬하고, 그에 따라 고유벡터도 정렬합니다. 이렇게 하면 가장 중요한 주성분부터 순서대로 선택할 수 있습니다.

5. **주성분 선택:** 가장 큰 k개의 고유값과 그에 따른 고유벡터를 선택하여 새로운 좌표계를 구성합니다. 여기서 k는 새로운 차원의 수로, 사용자가 원하는 차원의 수를 의미합니다.

6. **새로운 데이터 표현:** 선택된 주성분으로 원본 데이터를 새로운 좌표계로 변환합니다. 이로써 데이터의 차원이 감소하고, 주요한 정보가 유지되면서 노이즈나 불필요한 정보는 제거됩니다.

PCA는 주로 데이터 압축, 시각화, 노이즈 제거, 특성 선택 등의 목적으로 사용되며, 다양한 분야에서 활발하게 적용되고 있습니다.



# PCA 사용 목적

주성분 분석(Principal Component Analysis, PCA)는 다차원 데이터의 차원을 줄이고 주요한 정보를 보존하는 통계적 기법입니다. PCA의 목적은 원본 데이터의 특성을 최대한 유지하면서 새로운 좌표계로 데이터를 변환함으로써 차원을 축소하는 것입니다. 이는 다양한 상황에서 유용하게 활용될 수 있습니다.

1. **차원 감소 (Dimensionality Reduction):** 다차원 데이터는 각 차원이 서로 상호작용하며 데이터의 복잡성을 높일 수 있습니다. PCA를 사용하여 데이터를 더 낮은 차원으로 표현하면 계산 효율성이 향상되며, 시각화 및 해석이 쉬워집니다.

2. **데이터 시각화:** PCA는 데이터의 주요 구조를 보존하면서 차원을 감소시키므로, 데이터를 시각화하는 데 효과적입니다. 2차원 또는 3차원으로 축소된 데이터를 산점도 등으로 표현하여 데이터의 패턴이나 군집을 파악할 수 있습니다.

3. **데이터 전처리:** PCA는 데이터에서 불필요한 노이즈나 상관성이 낮은 특성을 제거함으로써 데이터의 정제된 형태를 얻을 수 있습니다. 이는 모델의 성능을 향상시키고, 과적합을 방지하는 데 도움이 됩니다.

4. **특성 선택:** PCA를 사용하면 주요한 정보를 가진 주성분들을 선택할 수 있습니다. 이를 통해 중요한 특성만을 고려하여 모델을 훈련시킬 수 있습니다.

5. **상관 관계 분석:** PCA는 데이터의 공분산 구조를 이용하기 때문에 다양한 변수 간의 상관 관계를 분석하는 데 활용됩니다.

6. **머신러닝에서의 성능 향상:** 차원 감소를 통해 모델의 계산 비용을 줄이고, 과적합을 방지하면서 모델의 일반화 성능을 향상시킬 수 있습니다.

PCA는 특히 고차원의 데이터나 다중 공선성이 있는 데이터에서 효과적으로 사용됩니다. 그러나 주의할 점은 PCA가 데이터의 특성을 변환하므로 변환 후의 주성분이 원래 데이터에서 어떤 의미를 갖는지를 이해하는 것이 중요합니다.

## PCA 활용 단계

1. 변수 수집
2. 변수별 정상성 테스트
3. 변수의 정상화
4. 정규화 (표준화)
5. PCA를 통해 산출된 변수의 변동성 설명력 확인
6. 분석에 사용될 변수의 수 결정

### 변수수집


In [ ]:
# 금리 스프레드 (Yield Spread)

start_dt, end_dt, tm_index, n = get_time_series_for_bok(1990, 1, 1, 30)
inter = 'M'

item_code1 = '817Y002'
item_code2 = ['010190000', '010195000', '010200000', '010200001', '010210000', '010300000', '010320000']
                #get_multi_econ_monthly_data(n, item_code1, item_code2, s_date, e_date, inter)
# bond_market_df = get_multi_econ_monthly_data(n, item_code1, item_code2, '', start_dt, end_dt, inter)

today = date.today()
past_date = date(1995, 1, 1)
date1 = today.strftime("%Y-%m-%d").replace('-', '')
date2 = past_date.strftime("%Y-%m-%d").replace('-', '')
s_date = date2
e_date = date1

days_diff = today - past_date
n = days_diff.days

daily_interest_df = get_bok_daily_data(n, item_code1, item_code2, s_date, e_date)

term_bond = daily_interest_df[(daily_interest_df['ITEM_NAME1'] == '국고채(10년)') | (daily_interest_df['ITEM_NAME1'] == '국고채(3년)')].copy()
bond_pivot = term_bond.pivot(index='TIME', columns='ITEM_NAME1', values='DATA_VALUE').dropna()
bond_pivot['term_spread'] = bond_pivot['국고채(10년)'] -  bond_pivot['국고채(3년)']
term_spread = bond_pivot.copy()
term_spread_re = term_spread.reset_index()
term_spread_re['TIME'] = pd.to_datetime(term_spread_re['TIME'], format='%Y-%m-%d')
term_spread_re

In [ ]:
# 기업경기실사지수(전망)

start_dt, end_dt, tm_index, n = get_time_series_for_bok(2003, 1, 1, 30)
inter = 'M'

item_code1 = '512Y008'
item_code2 = ['BA']
item_code3 = '99988'

indus_pros = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 종합주가지수

item_code1 = '902Y002'
item_code2 = ['3010101']
item_code3 = ''

kospi = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 소비자동향조사

item_code1 = '511Y002'
item_code2 = ['FME']
item_code3 = '99988'

csi = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 교역조건지수

item_code1 = '403Y005'
item_code2 = ['A']

trade = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 월말 대미환율

item_code1 = '731Y004'
item_code2 = ['0000001']
item_code3 = '0000100'

krw_dollar = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 국내건설수주액

item_code1 = '901Y020'
item_code2 = ['I42A']
item_code3 = ''

construct1 = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 주택매매가격지수

item_code1 = '901Y093'
item_code2 = ['H69A']
item_code3 = 'R70A'

h_price = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 기계수주액

item_code1 = '901Y018'
item_code2 = ['I351']
item_code3 = ''

machine = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

In [ ]:
# 소매 판매액 지수

item_code1 = '901Y100'
item_code2 = ['G0']
item_code3 = 'T3'

consumer_spending = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

### 정상성 테스트

In [ ]:
# 기업경기전망

# raw_data = indus_pros[['DATA_VALUE']].copy()
indus_pros['DATA_VALUE'] = indus_pros['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(indus_pros['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 종합주가지수

kospi['DATA_VALUE'] = kospi['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(kospi['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 소비자동향조사

csi['DATA_VALUE'] = csi['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(csi['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 교역조건지수

trade['DATA_VALUE'] = trade['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(trade['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 원달러 환율

krw_dollar['DATA_VALUE'] = krw_dollar['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(krw_dollar['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 국내건설수주액

construct1['DATA_VALUE'] = construct1['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(construct1['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 주택매매가격지수

h_price['DATA_VALUE'] = h_price['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(h_price['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 기계 수주액

machine['DATA_VALUE'] = machine['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(machine['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

In [ ]:
# 소매판매액 지수

consumer_spending['DATA_VALUE'] = consumer_spending ['DATA_VALUE'].astype('float64')

# ADF 테스트 수행
result = adfuller(consumer_spending['DATA_VALUE'])

# 결과 출력
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])

- 정상성 데이터 그룹 : 소비자동향조사, 기계수주액

- 비정상성 데이터 그룹 : 기업경기전망, 종합주가지수, 교역조건지수, 원달러환율, 국내건설수주액, 주택매매가격지수, 소매판매액지수

### 데이터 정상화 작업

In [ ]:
indus_pros['indus_prospect_diff'] = indus_pros['DATA_VALUE'].pct_change(12)
kospi['kospi_diff'] = kospi['DATA_VALUE'].pct_change()
trade['trade_diff'] = trade['DATA_VALUE'].pct_change()
krw_dollar['fx_diff'] = krw_dollar['DATA_VALUE'].pct_change()
construct1['construct_diff'] = construct1['DATA_VALUE'].pct_change()
h_price['house_price'] = h_price['DATA_VALUE'].pct_change()
consumer_spending['house_price_diff'] = consumer_spending['DATA_VALUE'].pct_change()

In [ ]:
csi['csi'] = csi['DATA_VALUE'].copy()

In [ ]:
machine['machine_order'] = machine['DATA_VALUE'].copy()

In [ ]:
macro_df = pd.concat([indus_pros['indus_prospect_diff'], kospi['kospi_diff'], trade['trade_diff'],
                      krw_dollar['fx_diff'], construct1['construct_diff'], h_price['house_price'],
                      consumer_spending['house_price_diff'], csi['csi'], machine['machine_order'],
                      ], axis=1)

In [ ]:
macro_df = macro_df.dropna()

In [ ]:
macro_df.index

In [ ]:
col_List = macro_df.columns.tolist()

for col in col_List:

    # ADF 테스트 수행
    result = adfuller(macro_df[col])

    # 결과 출력
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])

In [ ]:
start_dt, end_dt, tm_index, n = get_time_series_for_bok(1990, 1, 1, 30)
inter = 'M'

item_code1 = '301Y013'
item_code2 = ['2C1Y00', '2C2Y00']
item_code3 = ''

data = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)
data

### PCA를 위한 데이터 정규화

In [ ]:
##  library import
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [ ]:
std_scaler = preprocessing.StandardScaler().fit(macro_df)
scaled_macro = std_scaler.transform(macro_df)
scaled_macro_df = pd.DataFrame(scaled_macro, columns=macro_df.columns)

In [ ]:
scaled_macro_df

In [ ]:
scaled_macro_df.describe()

### 변수 총위험 설명력과 분석에 사용될 PCA 요인 결정

In [ ]:
pca = PCA(random_state=1107)
X_p = pca.fit_transform(scaled_macro_df)

variance_df = pd.Series(np.cumsum(pca.explained_variance_ratio_))
variance_df

In [ ]:
n = 4
pca = PCA(n_components=n)
macro_pca_df = pca.fit_transform(scaled_macro_df)
pca_col_name = ['macro_pca1', 'macro_pca2', 'macro_pca3', 'macro_pca4']
macro_pca_df = pd.DataFrame(macro_pca_df, columns=pca_col_name)

In [ ]:
macro_pca_df['date'] = macro_df.index
macro_pca_df.head(5)
external_vars = macro_pca_df.set_index('date')

external_vars_re = external_vars.loc['2010-09-30': '2023-09-30']
external_vars_re

### 분석 적용 사례

In [ ]:
start_dt, end_dt, tm_index, n = get_time_series_for_bok(1990, 1, 1, 30)
inter = 'M'

item_code1 = '301Y013'
item_code2 = ['2C1Y00', '2C2Y00']
item_code3 = ''

data = get_multi_econ_monthly_data(n, item_code1, item_code2, item_code3, start_dt, end_dt, inter)

data.columns= ['항목', '여행수지', '주기']

data_re = data.reset_index()

pivot_data = data_re.pivot(index='Date', columns='항목', values='여행수지')

# pivot_data['일반여행수입'].plot()

In [ ]:
pivot_data[['여행수입_log', '여행지급_log']] = np.log(pivot_data[['일반여행수입', '일반여행지급']])

In [ ]:
pivot_data_re = pivot_data.loc['2010-09-30':]
pivot_data_re

In [ ]:
from itertools import product
from tqdm.notebook import tqdm


tourism_balance_log = pivot_data_re[['여행수입_log', '여행지급_log']]
target_df = tourism_balance_log[['여행수입_log']].copy()

Y_train = target_df.iloc[:-1]

## Parameter Setting
p, q = range(0,2), range(0,2)
d = range(1,2)
P, Q = range(0,2), range(0,2)
D = range(0,2)
m = 12
trend_pdq = list(product(p, d, q))
seasonal_pdq = [(candi[0], candi[1], candi[2], m) for candi in list(product(P, D, Q))]

## SARIMAX
AIC = []
SARIMAX_order = []
for trend_param in tqdm(trend_pdq):
    for seasonal_params in seasonal_pdq:
        try:
            result =sm.tsa.SARIMAX(Y_train, trend='c',
                                   order=trend_param, seasonal_order=seasonal_params, exog=external_vars_re).fit()
            print('Fit SARIMAX: trend_order={} seasonal_order={} AIC={}, BIC={}'.format(trend_param, seasonal_params, result.aic, result.bic, end='\r'))
            AIC.append(result.aic)
            SARIMAX_order.append([trend_param, seasonal_params])
        except:
            continue
## Parameter Selection
print('The smallest AIC is {} for model SARIMAX{}x{}'.format(min(AIC), SARIMAX_order[AIC.index(min(AIC))][0], SARIMAX_order[AIC.index(min(AIC))][1]))

In [ ]:
# 리스트를 데이터프레임으로 변환
order_df = pd.DataFrame(SARIMAX_order, columns=['(p, d, q)', '(P, D, Q, M)'])
# (p, d, q) 컬럼을 나누어 각각의 컬럼으로 만들기
order_df[['p', 'd', 'q']] = pd.DataFrame(order_df['(p, d, q)'].tolist(), index=order_df.index)
order_df[['P', 'D', 'Q', 'M']] = pd.DataFrame(order_df['(P, D, Q, M)'].tolist(), index=order_df.index)

# 불필요한 열 제거
order_df = order_df.drop(['(p, d, q)', '(P, D, Q, M)'], axis=1)
order_df.head(5)

AIC_df = pd.DataFrame(AIC, columns= ['AIC'])

Sarima_AIC = pd.concat([order_df, AIC_df], axis=1).sort_values(by='AIC', ascending=True)
Sarima_AIC.head(3)

In [ ]:
seasonal_order = 12

step = 12

## SARIMAX
logarithm, differencing = True, False

fit_ts_sarimax = sm.tsa.SARIMAX(Y_train, order=(0,1,0), seasonal_order=(1,0,1,seasonal_order), trend='c', exog=external_vars_re).fit()
display(fit_ts_sarimax.summary())


주어진 결과로부터 다양한 통계적 특성을 확인할 수 있습니다. 주로 자기상관, 잔차의 정규성, 이분산성, 그리고 잔차의 자기회귀 여부 등을 확인하는 지표들이 포함되어 있습니다.

자기상관 (Autocorrelation):

Ljung-Box (L1) (Q): 0.48
Prob(Q): 0.49
Ljung-Box 통계량은 시계열 데이터에서 자기상관을 검정하는 통계량입니다. 여기서 Q 값이 낮으면 자기상관이 존재하지 않는 것으로 해석할 수 있습니다. Prob(Q)는 Q 값에 대한 p-value를 의미하며, 보통 0.05보다 작을 때 자기상관이 존재한다고 판단합니다. 이 경우에는 Prob(Q)가 0.49로 높으므로 자기상관이 존재하지 않는 것으로 해석할 수 있습니다.

잔차의 정규분포:

Jarque-Bera (JB): 14.79
Prob(JB): 0.00
Jarque-Bera 검정은 잔차의 정규성을 평가하는 테스트입니다. Prob(JB)는 검정 통계량에 대한 p-value를 나타냅니다. 여기서는 Prob(JB)가 0.00으로 매우 낮으므로, 잔차가 정규분포를 따르지 않는 것으로 해석할 수 있습니다.

이분산성 (Heteroskedasticity):

Heteroskedasticity (H): 1.46
Prob(H) (two-sided): 0.18
Heteroskedasticity 검정은 잔차의 분산이 일정하지 않은지를 확인합니다. Prob(H)가 0.18로 상당히 높기 때문에, 이분산성이 존재하지 않는 것으로 판단할 수 있습니다.

In [ ]:
# 훈련 데이터에 대한 예측
# predictions = fit_ts_sarimax.predict()
# predictions = predictions.iloc[1:]

# # 잔차 추출
# residuals = fit_ts_sarimax.resid
# residuals = residuals.iloc[1:]
# # Scatter plot 그리기
# plt.scatter(predictions, residuals,)
# plt.title('Residuals vs Predictions')
# plt.xlabel('Predictions')
# plt.ylabel('Residualss')
# plt.show()

# 글로벌 경기선행지수 수집

In [ ]:
# Composite Leading Indicator
# https://data.oecd.org/leadind/composite-leading-indicator-cli.htm
def getCLI(country):
    uri = 'https://stats.oecd.org/sdmx-json/data/DP_LIVE/' + country + '.CLI.AMPLITUD.LTRENDIDX.M/OECD?json-lang=en&dimensionAtObservation=allDimensions&startPeriod=2000-01&endPeriod=2023-10'
    resp = requests.get(uri)
    result = json.loads(resp.text)

    dates = []
    cli = []
    cli_code = 'CLI.' + country

    observations = result['dataSets'][0]['observations']
    for key in observations:
        obs = observations[key][0]
        cli.append(obs)

    time_period = result['structure']['dimensions']['observation'][5]['values']
    for date in time_period:
        date = date['id']
        year = int(date[:4])
        month = int(date[5:7])
        dates.append(datetime(year=year, month=month, day=1))

    df = pd.DataFrame(data=cli, index=dates, columns=[cli_code])
    return df

In [ ]:
country_code = 'KOR'

cli_code = 'CLI.' + country_code

cli = getCLI(country_code)

In [ ]:
cli